<a href="https://colab.research.google.com/github/klea0605/RI_projekat/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install mne

### Moduli

In [4]:
import numpy as np
import mne
from mne.decoding import CSP
import os

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier

In [6]:
os.chdir('/gdrive/MyDrive/RI')

In [7]:
import torch

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Moduli vezani za dataset

In [9]:
from Extract_data import Extract_data_from_subject
from TOL_dataset_utils import Select_time_window, Transform_for_classificator

### Inicijalizacija

In [10]:
# Classification parameters ---------------------------------------------------------------------

Subject_numbers = [1]
datatype = 'EEG'
Conditions = ['IN']  # samo inner speech klasifikujem ali moze i ostalo
Classes = [['All']] # all classes = left, right, up, down

root_dir = '.'
save_dir = './Results/'

# Preuzeto od Nieta

EEG specificnosti

In [12]:
sample_freq = 254
t_start = 1.5  # action interval pocinje ovde za svaki trial (bez obzira na condition)
t_end = 3.5   # relaxation interval ovde

channels_list = ["all"] # Generate features accross all eeg channels

# Filter signal band = [low_band , high_band, "band_name"]
# =============================================================================
bands = [
          (0.5, 4, 'Delta (0.5-4 Hz)')
         ,(4, 8, 'Theta (4-8 Hz)')
         ,(8, 12, 'Alpha (8-12 Hz)')
         ,(12, 20, 'Low Beta (12-20 Hz)')
         ,(20, 30, 'High Beta (20-30 Hz)')
         ,(30, 45, 'Low Gamma (30-45Hz)')
          ]

# Spatial Filter --------------------------------------------------------------
# Common Spatial Patterns: trazi u eeg signalima neke relevantnije signale; Verujemo ovom liku
n_components = 6
rank = None
reg = 'empirical'
log = True
norm_trace = False
cov_est = 'concat'
transform_into = 'average_power'

# Fituje se na filterovane podatke (kasnije)
csp = CSP(n_components=n_components,rank=rank, reg=reg, log=log, norm_trace = norm_trace, cov_est = cov_est, transform_into = transform_into)

In [13]:
scaler = MinMaxScaler()
results_save = True

### Funkcije

In [14]:
# filters X and returns filtered
def bandpass_filter(bands, sample_freq, X, device):

    X_return = []

    for band_number, band in enumerate(bands):
                freq_low = band[0]
                freq_high = band[1]
                X_filtered = mne.filter.filter_data(X, sample_freq, freq_low, freq_high, n_jobs = device) # brat je uzeo funkciju koja radi 2min i ponavlja je stalno na istom skupu ja nz


                # Stack features
                if band_number == 0:
                    X_return = X_filtered
                else:
                    X_return = np.hstack([X_return, X_filtered])

    return X_return

In [15]:
def csp_transform(data_train, y_train, data_test):
  Data_train = []
  Data_test = []

  Data_train = csp.fit_transform(data_train, y_train) # ? zbog ovoga nzm  kako da promenim ovu funkciju
  Data_test = csp.transform(data_test)

  return Data_train, Data_test

In [16]:
def save_results_for_participant(results_save, save_dir, Condition, subject, accuracy):
     if results_save:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        file_name = save_dir + "ACCURACY_TEST_CV_" + Condition  + "_Subject_" + str(subject)+ ".npy"

        np.save(file_name, accuracy)

## Proba

In [17]:
mock_x, mock_y = Extract_data_from_subject(root_dir, 1, datatype)
mock_x = Select_time_window(X = mock_x, t_start = t_start, t_end = t_end, fs = sample_freq)

In [18]:
mock_x

array([[[-9.20076058e-06, -1.30555659e-05, -1.22502267e-05, ...,
          1.54674673e-05,  1.01207836e-05,  1.66106374e-05],
        [-8.12596647e-06, -1.21511764e-05, -1.15074786e-05, ...,
          1.54690240e-05,  9.68146274e-06,  1.68052410e-05],
        [-9.34394261e-06, -1.23287744e-05, -1.07789176e-05, ...,
          1.52537283e-05,  1.02762168e-05,  1.64546244e-05],
        ...,
        [-1.38592406e-05, -1.21991260e-05, -1.71679119e-05, ...,
          1.80198643e-05,  2.08950247e-05,  1.38826268e-05],
        [ 1.91980734e-06,  6.70838512e-06,  3.11662562e-06, ...,
          3.91820644e-06,  1.37777326e-05, -5.83427020e-06],
        [ 2.05576241e-06,  6.88182136e-06,  2.72501478e-06, ...,
          2.88851471e-06,  1.30309223e-05, -5.51879617e-06]],

       [[ 4.04231999e-06,  2.28268734e-06,  3.23144830e-06, ...,
         -2.09964380e-05, -2.45920055e-05, -1.79599141e-05],
        [ 4.62390448e-06,  2.64458121e-06,  4.30135763e-06, ...,
         -2.10905993e-05, -2.53439927e

In [19]:
mock_x, mock_y = Transform_for_classificator(mock_x, mock_y, Conditions, Classes)

In [20]:
mock_y

array([1, 1, 3, 3, 3, 0, 2, 3, 3, 1, 0, 1, 3, 3, 1, 2, 0, 2, 0, 3, 1, 0,
       1, 1, 3, 2, 2, 0, 1, 2, 2, 0, 1, 2, 3, 0, 2, 0, 0, 2, 3, 0, 2, 0,
       1, 1, 3, 0, 2, 3, 3, 0, 2, 1, 1, 1, 3, 1, 1, 2, 3, 1, 1, 0, 2, 3,
       3, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2, 3, 3, 2, 1, 1, 2, 2, 3, 0, 0, 2,
       1, 1, 3, 1, 2, 0, 3, 0, 0, 0, 2, 1, 1, 1, 3, 2, 1, 2, 0, 1, 2, 3,
       2, 3, 3, 0, 3, 3, 2, 3, 0, 0, 1, 1, 3, 1, 0, 3, 2, 2, 0, 0, 0, 0,
       3, 0, 1, 2, 2, 2, 1, 2, 1, 2, 3, 3, 0, 3, 0, 0, 3, 1, 2, 1, 1, 0,
       3, 2, 3, 3, 1, 2, 2, 2, 1, 2, 0, 0, 1, 1, 3, 2, 1, 2, 0, 1, 3, 3,
       3, 3, 0, 0, 3, 0, 2, 1, 3, 3, 0, 1, 2, 2, 0, 0, 1, 3, 0, 3, 2, 1,
       2, 1])

In [21]:
print(mock_x.shape)
print(mock_y.shape)

(200, 128, 508)
(200,)


In [22]:
import random

In [23]:
indices = [i for i in range(200)]

In [24]:
train_indices = random.sample(indices, k = 150)
test_indices = list(set(indices).difference(train_indices))

In [ ]:
test_indices

In [27]:
mock_x = bandpass_filter(bands, 254, mock_x, device)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1677 samples (6.602 s)

CUDA not enabled in config, skipping initialization
CUDA not used, CUDA could not be initialized, falling back to n_jobs=None


<ipython-input-14-0f083eb7b2c8>:9: RuntimeWarning: filter_length (1677) is longer than the signal (508), distortion is likely. Reduce filter length or filter a longer signal.
  X_filtered = mne.filter.filter_data(X, sample_freq, freq_low, freq_high, n_jobs = device) # brat je uzeo funkciju koja radi 2min i ponavlja je stalno na istom skupu ja nz


Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 421 samples (1.657 s)

CUDA not enabled in config, skipping initialization
CUDA not used, CUDA could not be initialized, falling back to n_jobs=None
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequenc

Ovo ostavljam da demonstriram koliko dugo radi !

In [28]:
mock_x # hvala Bogu vise

array([[[-1.48214714e-07, -1.32448399e-06, -2.49423975e-06, ...,
          1.08673705e-06,  6.39286526e-07,  1.88784372e-07],
        [-2.15826258e-07, -1.43075482e-06, -2.63903508e-06, ...,
          1.47194811e-06,  8.67482313e-07,  2.58905639e-07],
        [-1.22292637e-07, -1.23401846e-06, -2.33953752e-06, ...,
          1.16716025e-06,  6.68652308e-07,  1.66573595e-07],
        ...,
        [ 5.13513724e-21,  9.27881694e-07,  2.31986043e-06, ...,
         -2.47251660e-06, -2.05237454e-06, -1.50877744e-21],
        [ 2.11758237e-22,  2.21092457e-06,  3.43399805e-06, ...,
         -1.10676087e-06, -1.17190553e-06, -9.52912066e-22],
        [ 6.35274710e-22,  2.92795463e-06,  4.22726442e-06, ...,
         -8.37028118e-07, -1.01676405e-06, -1.79994501e-21]],

       [[ 8.07219153e-07,  1.78333424e-06,  2.75528590e-06, ...,
         -1.77595119e-06, -1.27167054e-06, -7.63585713e-07],
        [ 7.43704508e-07,  1.76842588e-06,  2.78862692e-06, ...,
         -1.49122890e-06, -1.09812945e

In [29]:
mock_x.shape # isuse

(200, 768, 508)

### Klasifikacija

In [31]:
for subject in Subject_numbers:

    X, y = Extract_data_from_subject(root_dir, subject, datatype)
    # vreme kada postoji nadrazaj
    X = Select_time_window(X, t_start, t_end, sample_freq)
    # izvlacenje samo covert trial-a za sva 4 pravca
    X, y = Transform_for_classificator(X, y, Conditions, Classes)

    X = bandpass_filter(bands, sample_freq, X, device) # ! kad se ukloni device arg onda se izvrsava beskonacno

    # promenljive: n_splits
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)

    # Modeli za klasifikaciju
    models = {
        'random_forest': RandomForestClassifier()
    }

    # Hiperparametri za pretragu
    param_grids = {
    'random_forest': {
        'n_estimators': [10, 50],
        'max_depth': [None, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['auto', 'sqrt']
    }
    # },
    # 'conv1d': {
    #     'filters': [32, 64, 128],
    #     'kernel_size': [3, 5, 7],
    #     'strides': [1, 2],
    #     'activation': ['relu', 'tanh'],
    #     'dropout_rate': [0.0, 0.2, 0.4]
    # }
}

    # Dictionaries to store metrics
    cv_metrics_log = {model_name: {} for model_name in models}
    test_metrics = {model_name: [] for model_name in models}

    # Nested CV
    for train_valid_indices, test_indices in outer_cv.split(X, y):

        # Pretprocesiranje
        y_train_valid, y_test = y[train_valid_indices], y[test_indices]
        X_train_valid, X_test = csp_transform(X[train_valid_indices], y_train_valid, X[test_indices])
        X_train_valid = scaler.fit_transform(X_train_valid) # ? ovo me brine zbog cross validacije u grid_search-u
        X_test = scaler.transform(X_test)

        for model_name, model in models.items():
            # model.to(device)
            # Hyperparameter tuning using GridSearchCV                                                                     n_jobs - broj procesora
            grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name], scoring='accuracy', cv=inner_cv, n_jobs=2)

            # Cross validation on X_train_valid
            grid_search.fit(X_train_valid, y_train_valid) # ? ovo je problematicno mozda zbog bandpass filtera; isto kao za scaler

            # Atributi: best_estimator_, best_score_, best_params_
            best_params = grid_search.best_params_
            best_score = grid_search.best_score_

            cv_metrics_log[model_name] = {'Best params:': best_params,
                                          'Best CV score: ': best_score
                                          }

            # evaluacija na test skupu
            best_model = grid_search.best_estimator_
            best_model.fit(X_train_valid, y_train_valid)
            predictions = best_model.predict(X_test)

            test_metrics[model_name].append(accuracy_score(y_test, predictions))

            print(f'Subject: {subject} Test_metrics[{model_name}]: {test_metrics[model_name]}')

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1677 samples (6.602 s)

CUDA not enabled in config, skipping initialization
CUDA not used, CUDA could not be initialized, falling back to n_jobs=None


<ipython-input-14-0f083eb7b2c8>:9: RuntimeWarning: filter_length (1677) is longer than the signal (508), distortion is likely. Reduce filter length or filter a longer signal.
  X_filtered = mne.filter.filter_data(X, sample_freq, freq_low, freq_high, n_jobs = device) # brat je uzeo funkciju koja radi 2min i ponavlja je stalno na istom skupu ja nz


Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 421 samples (1.657 s)

CUDA not enabled in config, skipping initialization
CUDA not used, CUDA could not be initialized, falling back to n_jobs=None
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequenc

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Subject: 1 Test_metrics[random_forest]: [0.22388059701492538]
Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 768 dim * 1e+10  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0016 (2.2e-16 eps * 768 dim * 9.1e+09  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0016 (2.2e-16 eps * 768 dim * 9.2e+09  max singular value)
    Estimated rank (mag): 768
    MAG: rank 768 computed from 768 data channels with 0 projectors
Reducing data rank from 768 -> 768
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Us